In [1]:
import os
import zipfile
import time
import pickle
import gc

import tensorflow as tf 
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split


from feature_joint import addTime,addAd,addPosition,addAppCategories,addUserInfo
from _2_0_merge_others_features import add_new_trick,add_hist_cvr_smooth,add_smooth_pos_cvr,add_install2click
from _2_0_merge_others_features import add_cate_count,add_interval,add_basic_stats

from _2_1_gen_user_day_hour_click_feature import add_user_day_click_count,add_user_day_hour_count,add_user_click_stats,add_user_ratio_per_day,add_user_day_click
from _2_2_gen_app_install_feature import add_app_hist_install,add_app_start_installed,add_user_hist_install,add_user_start_installed_cateA
from _2_2_gen_app_install_feature import get_ConcatedAppIDTfidfVector_userinstalled

from _2_3_gen_IDs_count_per_day import add_IDs_count_per_day
from _2_4_gen_data_clk_conv_vector import get_ConcatedTfidfVector_ID_user_clicks,get_ConcatedTfidfVector_ID_user_convs
from _2_6_gen_combe_index import add_comb_features
from _2_8_gen_global_count import add_global_count_sum
from _2_9_gen_comb_cvr import add_comb_cvr

from utils import load_pickle,dump_pickle,get_feature_value,feature_spearmanr,feature_target_spearmanr,addCrossFeature,calibration
from utils import raw_data_path,trans_data_path,cache_pkl_path
from MeanEncoder import MeanEncoder

hhy_path = '../hhy/X_v2/'
%matplotlib inline

/home/shenweichen/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def training(clf,train_x,test_x,feature_names,cate_features,mode='offline'):
    if mode=='offline':
        start_time = time.time()
        clf.fit(train_x[feature_names],train_x['label'],
               eval_set=[(train_x[feature_names],train_x['label']),(test_x[feature_names],test_x['label'])],
                 feature_name=feature_names,categorical_feature=cate_features,
                    early_stopping_rounds=20,
                    verbose=10,
           )
        total_time = time.time()-start_time
        print('offline training done {0}m{1:.1f}s'.format(total_time//60,total_time%60))
        print('best iteration {0}'.format(clf.best_iteration))
        print('best score {0:.6f}'.format(clf.best_score['valid_1']['binary_logloss']))
        #pred = clf.predict_proba(test_x.loc[:,feature_names],num_iteration=clf.best_iteration)[:,1]
        #print('%.7f'%log_loss(test_x.loc[:,'label'],pred))
    
    elif mode=='online':
        start_time = time.time()
        clf.fit(train_x[feature_names],train_x['label'],
               eval_set=[(train_x[feature_names],train_x['label'])],
                 feature_name=feature_names,categorical_feature=cate_features,
                    #early_stopping_rounds=20,
                    verbose=10,
           )
        total_time = time.time()-start_time
        print('online training done {0}m{1:.1f}s'.format(total_time//60,total_time%60))
    return clf

def gen_result(clf,test_x,feature_names):
    #预测大概需要5分钟
    print('start predicting...')
    start_time = time.time()
    test_prob = clf.predict_proba(test_x[feature_names])[:,1]
    print(test_prob.mean())
    result = pd.read_csv('../result/demo_result.csv',index_col=['instanceID'])
    result['prob'] = test_prob
    filename = 'submission_'+'_'.join(time.ctime()[4:16].replace(':',' ').split(' '))+'.zip'
    cali_name = 'submission_'+'_'.join(time.ctime()[4:16].replace(':',' ').split(' '))+'.cali.zip'
    result.to_csv('submission.csv')
    print(filename)
    with zipfile.ZipFile('../result/'+filename, "w") as fout:
        fout.write("submission.csv", compress_type=zipfile.ZIP_DEFLATED)
    total_time = time.time() - start_time
    print('online predicting time {0}m{1:.1f}s'.format(total_time//60,total_time%60))
    return cali_name
    



def stratified_sampling(train,frac=0.33,seed=0):
    np.random.seed(seed)
    label_mean = train['label'].mean()
    pos_size = int(train.shape[0]*frac*label_mean)
    neg_size = int(train.shape[0]*frac*(1 - label_mean))
    pos_index = train[train.label==1].index
    neg_index = train[train.label==0].index
    sample_pos_idx = np.random.choice(pos_index,pos_size,replace=False)
    sample_neg_idx = np.random.choice(neg_index,neg_size,replace=False)
    sample_index = np.hstack([sample_pos_idx,sample_neg_idx])
    np.random.shuffle(sample_index)
    return train.loc[sample_index]

def addFeature(data):
    
    
    #pos_cvr = load_pickle('../trans_data_new/offline_pos_cvr.cvr')
    #data = pd.merge(data,pos_cvr,'left','positionID')
    
   
    tfidf_appcate_hour = get_TfidfVector_appCategory_user_action_hour()
    #data = pd.merge(data,tfidf_appcate_hour,'left','appCategory')
    
    n_components = 8
    svd = TruncatedSVD(n_components=n_components,random_state=0,)
    svd_vector = pd.DataFrame(svd.fit_transform(tfidf_appcate_hour.iloc[:,1:]),columns=['svd_appCategory_'+str(i) for i in range(n_components)],
                              index=tfidf_appcate_hour.appCategory).reset_index()
    #svd.explained_variance_ratio_.sum()
    data = pd.merge(data,svd_vector,'left','appCategory')
    
    return data


    

def addMeanEncoderFeature(split_train,split_test,feature,mode):
    train_feature_path = '../feature_pkl//'+mode+'_train_'+feature+'_MeanEncoder_.pkl'
    test_feature_path = '../feature_pkl/'+mode+'_test_'+feature+'_MeanEncoder_.pkl'
    feature_names = ['_'.join([feature,'pred','0']),'_'.join([feature,'pred','1'])]
    if os.path.exists(train_feature_path) and os.path.exists(test_feature_path):
        train_me = pickle.load(open(train_feature_path,'rb'))
        test_me = pickle.load(open(test_feature_path,'rb'))
        split_train = split_train.join(train_me)
        split_test = split_test.join(test_me)
    else:
        
        me = MeanEncoder([feature])
        split_train = me.fit_transform(split_train, split_train['label'])
        print(feature+' ME fit done')
        split_test_index = split_test.index
        split_test = me.transform(split_test)
        split_test.set_index(split_test_index,inplace=True)
        #pickle.dump(split_train[feature_names],open(train_feature_path,'wb'))
        #pickle.dump(split_test[feature_names],open(test_feature_path,'wb'))
                                                             
    return split_train,split_test


In [3]:
def load_data(start_day=23,end_day=26,load_test=False):
    """
    读取基本表拼接后的数据
    test表load_test = True
    """
    if load_test ==True:
        trans_train_path = trans_data_path+'trans_test_'+str(start_day)+'_'+str(end_day)+'.pkl'
        raw_train_path = raw_data_path +'test.pkl'
    else:
        trans_train_path = trans_data_path+'trans_train_'+str(start_day)+'_'+str(end_day)+'.pkl'
        raw_train_path = raw_data_path +'train.pkl'

    if os.path.exists(trans_train_path):
        print('found '+trans_train_path)
        train = pickle.load(open(trans_train_path,'rb'))
    else:
        print('generating '+trans_train_path)
        train = load_pickle(raw_train_path)

        train = addTime(train)
        train = train[(train.clickDay>=start_day)&(train.clickDay<=end_day)]
        train = addAd(train)
        train = addPosition(train)
        train = addAppCategories(train)
        train = addUserInfo(train)
             
        dump_pickle(train,trans_train_path)
    return train


def gen_online_data(train_start_day,train_end_day,test_day):
    
    train_x_path = cache_pkl_path +'online_train_x_'+str(train_start_day)+'_'+str(train_end_day)+'.pkl'
    test_x_path = cache_pkl_path + 'online_test_x_'+str(test_day)+'_'+str(test_day)+'.pkl'
    
    alpha = 0.647975342478
    beta = 34.83752176
    pos_na = alpha / (alpha + beta)
    
    
    
    if os.path.exists(train_x_path):
        print('found '+train_x_path)
        train_x  = load_pickle(train_x_path)
    else:
        print('generating '+train_x_path)
        train_x = load_data(train_start_day,train_end_day,False)
        train_x['age_cut']=pd.cut(train_x['age'],bins=[-1,0,18,25,35,45,55,65,np.inf],labels=False)
        
        #-----------trick-------------------------------
        train_x = add_new_trick(train_x,'train')
        #-----------intstall和action表相关----------------------
        print('adding install and actions...')
        train_x = add_user_start_installed_cateA(train_x)
        train_x = add_user_hist_install(train_x,'train')
        train_x = add_app_hist_install(train_x)
        train_x = add_app_start_installed(train_x)
         #-----------用户点击相关---------------------------- 
        print('adding user clicks...')
        train_x = add_user_day_click_count(train_x,['camgaignID','adID','sitesetID','appID',])
        train_x = add_user_day_hour_count(train_x,['camgaignID','adID','sitesetID','appID',])
        train_x = add_user_day_click(train_x)
        train_x = add_user_click_stats(train_x,'train')
         #------------转化率相关------------------------------
        print('adding conversions')
        train_x = add_smooth_pos_cvr(train_x,'online')
        train_x = train_x.fillna({'positionID_cvr_smooth': pos_na})
        train_x = add_hist_cvr_smooth(train_x,'train',cvr_list=[ 'creativeID', 'adID', 'appID','userID'])
         #--------------其他------------------------------------
        train_x = add_global_count_sum(train_x,31,stats_features=['positionID', 'creativeID', 'appID', 'adID', 'userID'])
        
        
        dump_pickle(train_x,train_x_path)
    
    if os.path.exists(test_x_path):
        print('found '+test_x_path)
        test_x  = load_pickle(test_x_path)
    else:
        print('generating '+test_x_path)
        test_x = load_data(test_day,test_day,True)
        test_x['age_cut']=pd.cut(test_x['age'],bins=[-1,0,18,25,35,45,55,65,np.inf],labels=False)
        
        #-----------trick-------------------------------
        test_x = add_new_trick(test_x,'test')
        #-----------intstall和action表相关----------------------
        print('adding install and actions...')
        test_x = add_user_start_installed_cateA(test_x)
        test_x = add_user_hist_install(test_x,'test')
        test_x = add_app_hist_install(test_x)
        test_x = add_app_start_installed(test_x)
         #-----------用户点击相关---------------------------- 
        print('adding user clicks...')
        test_x = add_user_day_click_count(test_x,['camgaignID','adID','sitesetID','appID',])
        test_x = add_user_day_hour_count(test_x,['camgaignID','adID','sitesetID','appID',])
        test_x = add_user_day_click(test_x)
        test_x = add_user_click_stats(test_x,'test')
         #------------转化率相关------------------------------
        print('adding conversions')
        test_x = add_smooth_pos_cvr(test_x,'online')
        test_x = test_x.fillna({'positionID_cvr_smooth': pos_na})
        test_x = add_hist_cvr_smooth(test_x,'test',cvr_list=[ 'creativeID', 'adID', 'appID','userID'])
         #--------------其他------------------------------------
        test_x = add_global_count_sum(test_x,31,stats_features=['positionID', 'creativeID', 'appID', 'adID', 'userID'])
       
        
        dump_pickle(test_x,test_x_path)
        
    return train_x,test_x
        

def gen_offline_data(train_start_day,train_end_day,test_day,):
    
    train_x_path = cache_pkl_path +'offline_train_x_'+str(train_start_day)+'_'+str(train_end_day)+'.pkl'
    test_x_path = cache_pkl_path + 'offline_test_x_'+str(test_day)+'_'+str(test_day)+'.pkl'
    if os.path.exists(train_x_path) and os.path.exists(test_x_path):
        print('found offline data')
        train_x = load_pickle(train_x_path)
        test_x = load_pickle(test_x_path)
    else:
        alpha = 0.640792339811
        beta = 34.2999347427
        pos_na = alpha / (alpha + beta)
        print('generating offline data')
        train_x = load_data(train_start_day,test_day,False)
        train_x['age_cut']=pd.cut(train_x['age'],bins=[-1,0,18,25,35,45,55,65,np.inf],labels=False)
            
        #-----------trick-------------------------------
        train_x = add_new_trick(train_x,'train')
        #-----------intstall和action表相关----------------------
        print('adding install and actions...')
        train_x = add_user_start_installed_cateA(train_x)
        train_x = add_user_hist_install(train_x,'train')
        train_x = add_app_hist_install(train_x)
        train_x = add_app_start_installed(train_x)
         #-----------用户点击相关---------------------------- 
        print('adding user clicks...')
        train_x = add_user_day_click_count(train_x,['camgaignID','adID','sitesetID','appID',])
        train_x = add_user_day_hour_count(train_x,['camgaignID','adID','sitesetID','appID',])
        train_x = add_user_day_click(train_x)
        train_x = add_user_click_stats(train_x,'train')
         #------------转化率相关------------------------------
        print('adding conversions')
        alpha = 0.640792339811
        beta = 34.2999347427
        pos_na = alpha / (alpha + beta)
        train_x = add_smooth_pos_cvr(train_x,'offline')
        train_x = train_x.fillna({'positionID_cvr_smooth': pos_na})
        train_x = add_hist_cvr_smooth(train_x,'train',cvr_list=[ 'creativeID', 'adID', 'appID','userID'])
         #--------------其他------------------------------------
        train_x = add_global_count_sum(train_x,27,stats_features=['positionID', 'creativeID', 'appID', 'adID', 'userID'])
        #------------分割train和test--------------------------
        print('splitting train and test ...')
        test_x = train_x[train_x.clickDay==test_day]
        train_x = train_x[(train_x.clickDay>=train_start_day)&(train_x.clickDay<=train_end_day)]
        dump_pickle(train_x,train_x_path,4)
        dump_pickle(test_x,test_x_path,4)
    return train_x,test_x

In [4]:
train_x,test_x = gen_offline_data(23,26,27)

found offline data


In [ ]:
def merge_ID_vector(split_train_x,split_test_x,ID_name,last_day,concated_list=['age_cut', 'gender', 'education', 'marriageStatus', 'haveBaby',],mode='local'):
    a = get_ConcatedTfidfVector_ID_user_clicks(ID_name,last_day,mode,concated_list=concated_list,drop_na=False)
    split_train_x = pd.merge(split_train_x,a,'left',ID_name)
    split_test_x = pd.merge(split_test_x,a,'left',ID_name)
    return split_train_x,split_test_x

In [4]:
train_x['age_cut']=pd.cut(train_x['age'],bins=[-1,0,18,25,35,45,55,65,np.inf],labels=False)
test_x['age_cut']=pd.cut(test_x['age'],bins=[-1,0,18,25,35,45,55,65,np.inf],labels=False)

In [5]:
COLUMNS = ['age_cut','gender', 'education', 'marriageStatus', 'haveBaby','appID','creativeID']

In [6]:
feature_names = [ 'age','creativeID', 'userID',
       'positionID', 'connectionType', 'telecomsOperator', 
       'adID', 'camgaignID', 'advertiserID', 'appID', 'appPlatform',
       'appCategory', 'sitesetID', 'positionType',   
       'gender', 'education', 'marriageStatus',
       'haveBaby', 'ht_province',  'rd_province',
                  'clickHour',
       'user_ad_click_day', 'user_camgaign_click_day', 'user_app_click_day',
       'user_site_click_day', 'user_click_day', 'user_ad_click_hour',
       'user_camgaign_click_hour', 'user_app_click_hour',
       'user_site_click_hour', 'installed_cate_0', 'installed_cate_1',
       'installed_cate_2', 'installed_cate_3', 'installed_cate_4',
       'installed_cate_5'
                ]# 'hometown','residence','time'
      #'score'
cate_features = ['gender', 'education', 'marriageStatus', 'haveBaby','clickHour',
                 'ht_province','rd_province','creativeID','adID', 
        'connectionType', 'telecomsOperator','sitesetID', 'positionType',#'userID',
               'camgaignID','appCategory','advertiserID','appPlatform','appID','positionID',
              
                #  'app_age_audiences',
              ]# 'time' 


try:
    feature_names.remove('clickTime')
except:
    pass

In [7]:
all_data = train_x.append(test_x)

In [8]:
lbe = LabelEncoder()

In [9]:
id_size_dict ={}
for feature in cate_features:
    all_data[feature] = lbe.fit_transform(all_data[feature])
    id_size_dict[feature] = all_data[feature].max()+1

In [10]:
train_x = all_data[all_data.clickDay<27]
test_x = all_data[all_data.clickDay==27]

In [11]:
del all_data
gc.collect()

448

In [12]:
id_size_list = []

In [13]:
for feature in cate_features:
    id_size_list.append(id_size_dict[feature])

In [14]:
def run(fetches,sess, X=None, y=None, ):
        feed_dict = {}
        if type(X) is list:
            for i in range(len(X)):
                feed_dict[id_inputs[i]] = X[i]
                #print(feed_dict[id_inputs[i]])
        else:
            feed_dict[id_inputs] = X
        if y is not None:
            feed_dict[label_inputs] = y
        return sess.run(fetches, feed_dict)

In [22]:
input_feature = cate_features#['appID','gender','connectionType']
embedding_size =500
id_dtype= tf.int32
id_field_num = len(input_feature)
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(0)
    global_step =  tf.Variable(0,dtype=tf.int32,trainable=False,name='global_step')

    with tf.name_scope('ID_input'):
        id_inputs =[tf.placeholder(id_dtype,shape=[None,]) for i in range(id_field_num)]
        label_inputs = tf.placeholder(tf.float32)
    
    with tf.name_scope('Embedding_matrix'):
        embeddings = [tf.Variable(tf.ones([id_size_list[i],embedding_size],)) for i in range(0,id_field_num)]
        #embedding1 = tf.Variable(tf.ones([id_size,embedding_size],))
        #embedding2 = tf.Variable(tf.ones([id_size,embedding_size],))
        #embedding3 = tf.Variable(tf.ones([id_size,embedding_size],))
        onehot_embeddings = [tf.Variable(tf.random_normal([id_size_list[i],1])) for i in range(0,id_field_num)]
        #onehot_embedding1 = tf.Variable(tf.random_normal([id_size,1]))
        #onehot_embedding2 = tf.Variable(tf.random_normal([id_size,1]))
        #onehot_embedding3 = tf.Variable(tf.random_normal([id_size,1]))
    
    with tf.name_scope('Embedding_layer'):
        embeds = [tf.nn.embedding_lookup(embeddings[i],id_inputs[i]) for i in range(0,id_field_num)]
        #embed1 = tf.nn.embedding_lookup(embedding1,appID)
        #embed2 = tf.nn.embedding_lookup(embedding2,genderID)
        #embed3 = tf.nn.embedding_lookup(embedding3,connectionTypeID)
        
        onehot_embeds = [tf.nn.embedding_lookup(onehot_embeddings[i],id_inputs[i]) for i in range(0,id_field_num)]
        #onehot_embed1 = tf.nn.embedding_lookup(onehot_embedding1,appID)
        #onehot_embed2 = tf.nn.embedding_lookup(onehot_embedding2,genderID)
        #onehot_embed3 = tf.nn.embedding_lookup(onehot_embedding3,connectionTypeID)
    
    with tf.name_scope('FM_layer'):
        fm_units = []
        embed_list=embeds#[embed1,embed2,embed3]
        for i in range(0,id_field_num):
            for j in range(i+1,id_field_num):
                temp = tf.reduce_sum(tf.multiply(embed_list[i],embed_list[j]),axis=1)
                fm_units.append(temp)

        sum_fm_units =tf.reduce_sum(fm_units,0)
        single_onehot_list = tf.concat(onehot_embeds,1)
        sum_linear_units = tf.reduce_sum(single_onehot_list,1)
    with tf.name_scope('Output'):
        logits =  sum_linear_units#tf.add(sum_fm_units,sum_linear_units)
        prob = tf.sigmoid(logits)
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=label_inputs,logits=logits))
        batch_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=label_inputs,logits=logits))
        #optimizer = tf.train.AdamOptimizer().minimize(loss,global_step=global_step)
        optimizer = tf.train.FtrlOptimizer(0.05).minimize(loss,global_step=global_step)
    with tf.name_scope('Summary'):
        tf.summary.scalar('loss', loss)
        summary_op = tf.summary.merge_all()

In [18]:
input_feature = cate_features

In [27]:
early_stop_round = 20
min_loss = 1e9
chance_round = 0

In [ ]:
train_size = train_x.shape[0]
valid_size = test_x.shape[0]
batch_size = 1024#10240
valid_batch_size = 102400
batch_num = train_size//batch_size
valid_batch_num = valid_size//valid_batch_size
num_round = 1000
va_input_X = [test_x[feature].values for feature in input_feature]
va_input_y = test_x['label'].values

with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./deepFM_log/',graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    for i in range(num_round):
        for j in tqdm(range(0,1+batch_num)): 
            X_i = train_x.iloc[j*batch_size:min((j+1)*batch_size,train_size)]
            input_X = [X_i[feature].values for feature in input_feature]#X_i[input_feature]
            #print(type(input_X.values))
            input_y =X_i['label'].values
            #_,l,summary = sess.run([optimizer,loss,summary_op],feed_dict={id_inputs:input_X.values,y:input_y})
            _,l,summary = run([optimizer,loss,summary_op],sess=sess,X = input_X,y = input_y)
            #logit = run([logits],sess=sess,X = input_X,y = input_y)
            #print(l)
        va_loss_sum = 0 
        for j in range(0,1+valid_batch_num):
            X_i = test_x.iloc[j*valid_batch_size:min((j+1)*valid_batch_size,valid_size)]
            input_X = [X_i[feature].values for feature in input_feature]#X_i[input_feature]
            input_y =X_i['label'].values
            va_batch_loss = run([batch_loss,],sess=sess,X=input_X,y=input_y,)
            #print(va_batch_loss)
            va_loss_sum += va_batch_loss[0]
        va_loss = va_loss_sum/valid_size
        print('epoch {0} va-loss {1:.7f}'.format(i,va_loss))
        if va_loss < min_loss:
            min_loss = va_loss
            chance_round = 0
        else:
            chance_round +=1
        if va_loss < 0.105 or chance_round > early_stop_round:
            print('done')
            saver.save(sess,'./LR_log/save_model',global_step=global_step)
            break
        saver.save(sess,'./LR_log/save_model',global_step=global_step)       

100%|██████████| 10354/10354 [00:59<00:00, 174.84it/s]


epoch 0 va-loss 0.1138438


100%|██████████| 10354/10354 [01:04<00:00, 160.45it/s]


epoch 1 va-loss 0.1120360


100%|██████████| 10354/10354 [01:00<00:00, 170.42it/s]


...

100%|██████████| 10354/10354 [01:02<00:00, 166.54it/s]


epoch 22 va-loss 0.1076165


100%|██████████| 10354/10354 [01:02<00:00, 166.62it/s]


epoch 23 va-loss 0.1075744


 18%|█▊        | 1896/10354 [00:11<00:48, 173.69it/s]

In [30]:
va_loss

0.10603944895273673